In [1]:
# This notebook shows how to practice 
# different optimization techniques 
# with the heat equation solver
#
# Prepared: Octavian Andrei, 2020 

## References

1. The code for this exercise is located [here](https://github.com/csc-training/hpc-python/tree/master/cython/heat-equation).

2. [Cython user guide for numpy](https://cython.readthedocs.io/en/latest/src/userguide/numpy_tutorial.html)

# Optimising heat equation with Cython

### Creating a Cython extension

Write a `setup.py` for creating a Cython version of [heat.py](https://github.com/csc-training/hpc-python/blob/master/cython/heat-equation/heat.py)
module, and use it from the main program [heat_main.py](https://github.com/csc-training/hpc-python/blob/master/cython/heat-equation/heat_main.py).
How much does simple Cythonization (i.e. diminishing the interpreting
overhead) improve the performance?

### Optimising

Based on the profile in the performance measurement
[exercise](https://github.com/csc-training/hpc-python/blob/master/performance/cprofile) optimise the most time
consuming part of the algorithm. If you did not finish the profiling
exercise, you can look at example profile [here](https://github.com/csc-training/hpc-python/blob/master/cython/heat-equation/profile.md). 

Utilize all the tricks you have learned so far (type declarations,
fast array indexing, compiler directives, C functions, ...).

Investigate how the different optimizations affect the performance. You
can use applications own timers and/or **timeit**. Annotated HTML-report with
`cython -a …` can be useful when tuning performance.

When finished with the optimisation, compare performance to
Python/NumPy model solution (in
[numpy/heat-equation](https://github.com/csc-training/hpc-python/blob/master/numpy/heat-equation)), which uses array 
operations. You can play around also with larger input data as provided in
[bottle_medium.dat](https://github.com/csc-training/hpc-python/blob/master/cython/heat-equation/bottle_medium.dat) and [bottle_large.dat](https://github.com/csc-training/hpc-python/blob/master/cython/heat-equation/bottle_large.dat).



In [2]:
from __future__ import print_function
%load_ext cython
import Cython
print(Cython.__version__)

0.29.15


In [3]:
import numpy as np
import matplotlib
import time
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# Set the colormap
plt.rcParams['image.cmap'] = 'BrBG'

In [4]:
# dictionary to store the execution times
timeit_dict = dict()      # bottle.dat
timeit_middle = dict()    # bottle_medium.dat
timeit_large = dict()     # bottle_large.dat

In [5]:
# original code
# %load '../cython/heat-equation/heat.py'
# %load '../cython/heat-equation/heat_main.py'

def evolve(u, u_previous, a, dt, dx2, dy2):
    """Explicit time evolution.
       u:            new temperature field
       u_previous:   previous field
       a:            diffusion constant
       dt:           time step. """

    n, m = u.shape

    for i in range(1, n-1):
        for j in range(1, m-1):
            u[i, j] = u_previous[i, j] + a * dt * ( \
             (u_previous[i+1, j] - 2*u_previous[i, j] + \
              u_previous[i-1, j]) / dx2 + \
             (u_previous[i, j+1] - 2*u_previous[i, j] + \
                 u_previous[i, j-1]) / dy2 )
    u_previous[:] = u[:]

def iterate(field, field0, a, dx, dy, timesteps, image_interval):
    """Run fixed number of time steps of heat equation"""

    dx2 = dx**2
    dy2 = dy**2

    # For stability, this is the largest interval possible
    # for the size of the time-step:
    dt = dx2*dy2 / ( 2*a*(dx2+dy2) )    

    for m in range(1, timesteps+1):
        evolve(field, field0, a, dt, dx2, dy2)
        if m % image_interval == 0:
            write_field(field, m)

def init_fields(filename):
    # Read the initial temperature field from file
    field = np.loadtxt(filename)
    field0 = field.copy() # Array for field of previous time step
    return field, field0

def write_field(field, step):
    plt.gca().clear()
    plt.imshow(field)
    plt.axis('off')
    plt.savefig('heat_{0:03d}.png'.format(step))


def main(input_file='bottle.dat', a=0.5, dx=0.1, dy=0.1, 
         timesteps=200, image_interval=4000):

    # Initialise the temperature field
    field, field0 = init_fields(input_file)

    print("Heat equation solver")
    # print("Diffusion constant: {}".format(a))
    print("Input file: {}".format(input_file))
    # print("Parameters")
    # print("----------")
    # print("  nx={} ny={} dx={} dy={}".format(field.shape[0], field.shape[1],
    #                                          dx, dy))
    # print("  time steps={}  image interval={}".format(timesteps,
    #                                                      image_interval))

    # Plot/save initial field
    write_field(field, 0)
    # Iterate
    t0 = time.time()
    iterate(field, field0, a, dx, dy, timesteps, image_interval)
    t1 = time.time()
    # Plot/save final field
    write_field(field, timesteps)

    print("Simulation finished in {0} s".format(t1-t0))
    
main()

Heat equation solver
Input file: bottle.dat
Simulation finished in 38.91359806060791 s


In [6]:
timeit_result = %timeit -n1 -r 3 -o main()

Heat equation solver
Input file: bottle.dat
Simulation finished in 40.77020788192749 s
Heat equation solver
Input file: bottle.dat
Simulation finished in 41.60613298416138 s
Heat equation solver
Input file: bottle.dat
Simulation finished in 38.71163487434387 s
40.6 s ± 1.22 s per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [7]:
original_time = timeit_result.average
timeit_dict['Original python'] = original_time
original_time

40.592430594

In [8]:
# it takes ~7X longer than the small
# bottle_medium.dat
timeit_result = %timeit -n1 -r 1 -o main(input_file='bottle_medium.dat')
run_time = timeit_result.average
print(run_time)
timeit_middle['Original python'] = run_time

Heat equation solver
Input file: bottle_medium.dat
Simulation finished in 282.35807394981384 s
4min 42s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
282.861264109


    Heat equation solver
    Input file: bottle_medium.dat
    Simulation finished in 251.13387298583984 s
    4min 11s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)

In [9]:
# it takes ~31X longer than the small
# bottle_large.dat
timeit_result = %timeit -n1 -r 1 -o main(input_file='bottle_large.dat')
run_time = timeit_result.average
print(run_time)
timeit_large['Original python'] = run_time

Heat equation solver
Input file: bottle_large.dat
Simulation finished in 1044.8263030052185 s
17min 26s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
1046.111133612


    Heat equation solver
    Input file: bottle_large.dat
    Simulation finished in 1114.454592704773 s
    18min 35s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Numpy version

In [10]:
def evolve(u, u_previous, a, dt, dx2, dy2):
    """Explicit time evolution.
       u:            new temperature field
       u_previous:   previous field
       a:            diffusion constant
       dt:           time step
       dx2:          grid spacing squared, i.e. dx^2
       dy2:            -- "" --          , i.e. dy^2"""

    u[1:-1, 1:-1] = u_previous[1:-1, 1:-1] + a * dt * ( \
            (u_previous[2:, 1:-1] - 2*u_previous[1:-1, 1:-1] + \
             u_previous[:-2, 1:-1]) / dx2 + \
            (u_previous[1:-1, 2:] - 2*u_previous[1:-1, 1:-1] + \
                 u_previous[1:-1, :-2]) / dy2 )
    u_previous[:] = u[:]

def iterate(field, field0, a, dx, dy, timesteps, image_interval):
    """Run fixed number of time steps of heat equation"""

    dx2 = dx**2
    dy2 = dy**2

    # For stability, this is the largest interval possible
    # for the size of the time-step:
    dt = dx2*dy2 / ( 2*a*(dx2+dy2) )    

    for m in range(1, timesteps+1):
        evolve(field, field0, a, dt, dx2, dy2)
        if m % image_interval == 0:
            write_field(field, m)

def init_fields(filename):
    # Read the initial temperature field from file
    field = np.loadtxt(filename)
    field0 = field.copy() # Array for field of previous time step
    return field, field0

def write_field(field, step):
    plt.gca().clear()
    plt.imshow(field)
    plt.axis('off')
    plt.savefig('heat_{0:03d}.png'.format(step))

def main(input_file='bottle.dat', a=0.5, dx=0.1, dy=0.1, 
         timesteps=200, image_interval=4000):

    # Initialise the temperature field
    field, field0 = init_fields(input_file)

    print("Heat equation solver")
    # print("Diffusion constant: {}".format(a))
    # print("Input file: {}".format(input_file))
    # print("Parameters")
    # print("----------")
    # print("  nx={} ny={} dx={} dy={}".format(field.shape[0], field.shape[1],
    #                                          dx, dy))
    # print("  time steps={}  image interval={}".format(timesteps,
    #                                                      image_interval))

    # Plot/save initial field
    write_field(field, 0)
    # Iterate
    t0 = time.time()
    iterate(field, field0, a, dx, dy, timesteps, image_interval)
    t1 = time.time()
    # Plot/save final field
    write_field(field, timesteps)

    print("Simulation finished in {0} s".format(t1-t0))

main()

Heat equation solver
Simulation finished in 0.17605018615722656 s


In [11]:
timeit_result = %timeit -n1 -r 3 -o main()

Heat equation solver
Simulation finished in 0.19539403915405273 s
Heat equation solver
Simulation finished in 0.18122482299804688 s
Heat equation solver
Simulation finished in 0.18755602836608887 s
422 ms ± 29 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


In [12]:
np_time = timeit_result.average
timeit_dict['Numpy'] = np_time
print(np_time)

0.42193463800003883


In [13]:
# Original vs. numpy. any improvement?
print(original_time/np_time)

96.2054947335143


In [14]:
# bottle_medium.dat
timeit_result = %timeit -n1 -r 1 -o main(input_file='bottle_medium.dat')
run_time = timeit_result.average
timeit_middle['Numpy'] = run_time
print(timeit_middle)

Heat equation solver
Simulation finished in 1.897078037261963 s
2.32 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
{'Original python': 282.861264109, 'Numpy': 2.322825343999966}


In [15]:
# bottle_large.dat
timeit_result = %timeit -n1 -r 1 -o main(input_file='bottle_large.dat')
run_time = timeit_result.average
timeit_large['Numpy'] = run_time
print(timeit_large)

Heat equation solver
Simulation finished in 7.9606311321258545 s
9.15 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
{'Original python': 1046.111133612, 'Numpy': 9.146373484999913}


# Helper functions

In [16]:
def compare_time(current, reference, name):
    ratio = reference/current
    if ratio > 1:
        word = "faster"
    else:
        ratio = 1 / ratio 
        word = "slower"
        
    print("We are", "{0:.1f}".format(ratio), "times", word, "than the", name, "version.")

def print_report(main_function, **kwargs):
    timeit_result = %timeit -n1 -r 3 -o main_function(**kwargs)
    run_time = timeit_result.average
    timeit_dict['Optimized'] = run_time
    print('\nCurrent run_time: {:.10f} s'.format(run_time))
    compare_time(run_time, original_time, "original Python")
    compare_time(run_time, np_time, "NumPy")

# Python version compiled with Cython

In [17]:
%%cython -a
import time
import numpy as np
import matplotlib.pyplot as plt

def evolve(u, u_previous, a, dt, dx2, dy2):
    """Explicit time evolution.
       u:            new temperature field
       u_previous:   previous field
       a:            diffusion constant
       dt:           time step. """

    n, m = u.shape

    for i in range(1, n-1):
        for j in range(1, m-1):
            u[i, j] = u_previous[i, j] + a * dt * ( \
             (u_previous[i+1, j] - 2*u_previous[i, j] + \
              u_previous[i-1, j]) / dx2 + \
             (u_previous[i, j+1] - 2*u_previous[i, j] + \
                 u_previous[i, j-1]) / dy2 )
    u_previous[:] = u[:]

def iterate(field, field0, a, dx, dy, timesteps, image_interval):
    """Run fixed number of time steps of heat equation"""

    dx2 = dx**2
    dy2 = dy**2

    # For stability, this is the largest interval possible
    # for the size of the time-step:
    dt = dx2*dy2 / ( 2*a*(dx2+dy2) )    

    for m in range(1, timesteps+1):
        evolve(field, field0, a, dt, dx2, dy2)
        if m % image_interval == 0:
            write_field(field, m)

def init_fields(filename):
    # Read the initial temperature field from file
    field = np.loadtxt(filename)
    field0 = field.copy() # Array for field of previous time step
    return field, field0

def write_field(field, step):
    plt.gca().clear()
    plt.imshow(field)
    plt.axis('off')
    plt.savefig('heat_{0:03d}.png'.format(step))


def main(input_file='bottle.dat', a=0.5, dx=0.1, dy=0.1, 
         timesteps=200, image_interval=4000):

    # Initialise the temperature field
    field, field0 = init_fields(input_file)

    print("Heat equation solver")
    # print("Diffusion constant: {}".format(a))
    # print("Input file: {}".format(input_file))
    # print("Parameters")
    # print("----------")
    # print("  nx={} ny={} dx={} dy={}".format(field.shape[0], field.shape[1],
    #                                          dx, dy))
    # print("  time steps={}  image interval={}".format(timesteps,
    #                                                      image_interval))

    # Plot/save initial field
    write_field(field, 0)
    # Iterate
    t0 = time.time()
    iterate(field, field0, a, dx, dy, timesteps, image_interval)
    t1 = time.time()
    # Plot/save final field
    write_field(field, timesteps)

    print("Simulation finished in {0} s".format(t1-t0))

In [18]:
# let's check what are the improvements
print_report(main)

Heat equation solver
Simulation finished in 28.7319757938385 s
Heat equation solver
Simulation finished in 29.871647119522095 s
Heat equation solver
Simulation finished in 28.884845972061157 s
29.4 s ± 507 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)

Current run_time: 29.3618392157 s
We are 1.4 times faster than the original Python version.
We are 69.6 times slower than the NumPy version.


In [19]:
# add this step to the dict
timeit_dict['Compiled with cython'] = timeit_dict['Optimized']
print(timeit_dict)

{'Original python': 40.592430594, 'Numpy': 0.42193463800003883, 'Optimized': 29.361839215666578, 'Compiled with cython': 29.361839215666578}


# Adding types:

In [20]:
# add variable types (change from Python-like object to C-like)
# int: n, m
# double: a, dt, dx2, dy2

In [21]:
%%cython
import time
import numpy as np
import matplotlib.pyplot as plt

def evolve(u, u_previous, double a, double dt, double dx2, double dy2):
    """Explicit time evolution.
       u:            new temperature field
       u_previous:   previous field
       a:            diffusion constant
       dt:           time step. """

    cdef int n = u.shape[0]
    cdef int m = u.shape[1]

    for i in range(1, n-1):
        for j in range(1, m-1):
            u[i, j] = u_previous[i, j] + a * dt * ( \
             (u_previous[i+1, j] - 2*u_previous[i, j] + \
              u_previous[i-1, j]) / dx2 + \
             (u_previous[i, j+1] - 2*u_previous[i, j] + \
                 u_previous[i, j-1]) / dy2 )
    u_previous[:] = u[:]

def iterate(field, field0, a, dx, dy, timesteps, image_interval):
    """Run fixed number of time steps of heat equation"""

    dx2 = dx**2
    dy2 = dy**2

    # For stability, this is the largest interval possible
    # for the size of the time-step:
    dt = dx2*dy2 / ( 2*a*(dx2+dy2) )    

    for m in range(1, timesteps+1):
        evolve(field, field0, a, dt, dx2, dy2)
        if m % image_interval == 0:
            write_field(field, m)

def init_fields(filename):
    # Read the initial temperature field from file
    field = np.loadtxt(filename)
    field0 = field.copy() # Array for field of previous time step
    return field, field0

def write_field(field, step):
    plt.gca().clear()
    plt.imshow(field)
    plt.axis('off')
    plt.savefig('heat_{0:03d}.png'.format(step))


def main(input_file='bottle.dat', a=0.5, dx=0.1, dy=0.1, 
         timesteps=200, image_interval=4000):

    # Initialise the temperature field
    field, field0 = init_fields(input_file)

    print("Heat equation solver")
    # print("Diffusion constant: {}".format(a))
    # print("Input file: {}".format(input_file))
    # print("Parameters")
    # print("----------")
    # print("  nx={} ny={} dx={} dy={}".format(field.shape[0], field.shape[1],
    #                                          dx, dy))
    # print("  time steps={}  image interval={}".format(timesteps,
    #                                                      image_interval))

    # Plot/save initial field
    write_field(field, 0)
    # Iterate
    t0 = time.time()
    iterate(field, field0, a, dx, dy, timesteps, image_interval)
    t1 = time.time()
    # Plot/save final field
    write_field(field, timesteps)

    print("Simulation finished in {0} s".format(t1-t0))

In [22]:
# let's check what are the improvements
print_report(main)

Heat equation solver
Simulation finished in 32.0338408946991 s
Heat equation solver
Simulation finished in 33.537384033203125 s
Heat equation solver
Simulation finished in 34.14824986457825 s
33.5 s ± 897 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)

Current run_time: 33.4531745733 s
We are 1.2 times faster than the original Python version.
We are 79.3 times slower than the NumPy version.


In [23]:
# add this step to the dict
timeit_dict['Optimized typed'] = timeit_dict['Optimized']
print(timeit_dict)

{'Original python': 40.592430594, 'Numpy': 0.42193463800003883, 'Optimized': 33.45317457333332, 'Compiled with cython': 29.361839215666578, 'Optimized typed': 33.45317457333332}


# C-function, array indexing using memoryviews

In [24]:
# u, u_previous are still NumPy arrays, so Python objects!
# We solve this aspect by using memoryviews
# double[:,:] u, u_previous
# c-function

In [25]:
%%cython
import time
import numpy as np
import matplotlib.pyplot as plt

# cdef means here that this function is a plain C function (so faster).
# To get all the benefits, we type the arguments.
# Use 2D memoryview
cdef evolve(double[:,:] u, double[:,:] u_previous, 
            double a, double dt, double dx2, double dy2):
    """Explicit time evolution.
       u:            new temperature field
       u_previous:   previous field
       a:            diffusion constant
       dt:           time step. """

    # The "cdef" keyword is also used within functions to type variables. 
    # It can only be used at the top indentation level.
    cdef int n = u.shape[0]
    cdef int m = u.shape[1]
    
    for i in range(1, n-1):
        for j in range(1, m-1):
            u[i, j] = u_previous[i, j] + a * dt * ( \
             (u_previous[i+1, j] - 2*u_previous[i, j] + \
              u_previous[i-1, j]) / dx2 + \
             (u_previous[i, j+1] - 2*u_previous[i, j] + \
                 u_previous[i, j-1]) / dy2 )
    u_previous[:] = u[:]

def iterate(field, field0, a, dx, dy, timesteps, image_interval):
    """Run fixed number of time steps of heat equation"""

    dx2 = dx**2
    dy2 = dy**2

    # For stability, this is the largest interval possible
    # for the size of the time-step:
    dt = dx2*dy2 / ( 2*a*(dx2+dy2) )    

    for m in range(1, timesteps+1):
        evolve(field, field0, a, dt, dx2, dy2)
        if m % image_interval == 0:
            write_field(field, m)

def init_fields(filename):
    # Read the initial temperature field from file
    field = np.loadtxt(filename)
    field0 = field.copy() # Array for field of previous time step
    return field, field0

def write_field(field, step):
    plt.gca().clear()
    plt.imshow(field)
    plt.axis('off')
    plt.savefig('heat_{0:03d}.png'.format(step))


def main(input_file='bottle.dat', a=0.5, dx=0.1, dy=0.1, 
         timesteps=200, image_interval=4000):

    # Initialise the temperature field
    field, field0 = init_fields(input_file)

    print("Heat equation solver")
    # print("Diffusion constant: {}".format(a))
    # print("Input file: {}".format(input_file))
    # print("Parameters")
    # print("----------")
    # print("  nx={} ny={} dx={} dy={}".format(field.shape[0], field.shape[1],
    #                                          dx, dy))
    # print("  time steps={}  image interval={}".format(timesteps,
    #                                                      image_interval))

    # Plot/save initial field
    write_field(field, 0)
    # Iterate
    t0 = time.time()
    iterate(field, field0, a, dx, dy, timesteps, image_interval)
    t1 = time.time()
    # Plot/save final field
    write_field(field, timesteps)

    print("Simulation finished in {0} s".format(t1-t0))

In [26]:
# let's check what are the improvements
print_report(main)

Heat equation solver
Simulation finished in 0.7121129035949707 s
Heat equation solver
Simulation finished in 0.6335551738739014 s
Heat equation solver
Simulation finished in 0.610313892364502 s
880 ms ± 51.8 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)

Current run_time: 0.8803729133 s
We are 46.1 times faster than the original Python version.
We are 2.1 times slower than the NumPy version.


In [27]:
# add this step to the dict
timeit_dict['Optimized memoryview'] = timeit_dict['Optimized']
print(timeit_dict)

{'Original python': 40.592430594, 'Numpy': 0.42193463800003883, 'Optimized': 0.8803729133333794, 'Compiled with cython': 29.361839215666578, 'Optimized typed': 33.45317457333332, 'Optimized memoryview': 0.8803729133333794}


# Declaring the Numpy arrays as contiguous + decorators

In [28]:
# Compiler directives
# Declare the memoryview as contiguous
# Deactivate bounds checking
# Deactivate negative indexing

In [29]:
%%cython
import time
import numpy as np
import matplotlib.pyplot as plt
import cython

@cython.boundscheck(False)  # bounds
@cython.wraparound(False)   # negative
@cython.profile(True)
cdef evolve(double[:,::1] u, double[:,::1] u_previous, 
            double a, double dt, double dx2, double dy2):
    """Explicit time evolution.
       u:            new temperature field
       u_previous:   previous field
       a:            diffusion constant
       dt:           time step. """

    cdef Py_ssize_t n = u.shape[0]
    cdef Py_ssize_t m = u.shape[1]
    
    cdef Py_ssize_t i,j
    
    for i in range(1, n-1):
        for j in range(1, m-1):
            u[i, j] = u_previous[i, j] + a * dt * ( \
             (u_previous[i+1, j] - 2*u_previous[i, j] + \
              u_previous[i-1, j]) / dx2 + \
             (u_previous[i, j+1] - 2*u_previous[i, j] + \
                 u_previous[i, j-1]) / dy2 )
    u_previous[:] = u[:]

def iterate(field, field0, a, dx, dy, timesteps, image_interval):
    """Run fixed number of time steps of heat equation"""

    dx2 = dx**2
    dy2 = dy**2

    # For stability, this is the largest interval possible
    # for the size of the time-step:
    dt = dx2*dy2 / ( 2*a*(dx2+dy2) )    

    for m in range(1, timesteps+1):
        evolve(field, field0, a, dt, dx2, dy2)
        if m % image_interval == 0:
            write_field(field, m)

def init_fields(filename):
    # Read the initial temperature field from file
    field = np.loadtxt(filename)
    field0 = field.copy() # Array for field of previous time step
    return field, field0

def write_field(field, step):
    plt.gca().clear()
    plt.imshow(field)
    plt.axis('off')
    plt.savefig('heat_{0:03d}.png'.format(step))


def main(input_file='bottle.dat', a=0.5, dx=0.1, dy=0.1, 
         timesteps=200, image_interval=4000):

    # Initialise the temperature field
    field, field0 = init_fields(input_file)

    print("Heat equation solver")
    # print("Diffusion constant: {}".format(a))
    # print("Input file: {}".format(input_file))
    # print("Parameters")
    # print("----------")
    # print("  nx={} ny={} dx={} dy={}".format(field.shape[0], field.shape[1],
    #                                          dx, dy))
    # print("  time steps={}  image interval={}".format(timesteps,
    #                                                      image_interval))

    # Plot/save initial field
    write_field(field, 0)
    # Iterate
    t0 = time.time()
    iterate(field, field0, a, dx, dy, timesteps, image_interval)
    t1 = time.time()
    # Plot/save final field
    write_field(field, timesteps)

    print("Simulation finished in {0} s".format(t1-t0))

In [30]:
print_report(main)

Heat equation solver
Simulation finished in 0.04173016548156738 s
Heat equation solver
Simulation finished in 0.041323184967041016 s
Heat equation solver
Simulation finished in 0.04307413101196289 s
290 ms ± 12.2 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)

Current run_time: 0.2900572280 s
We are 139.9 times faster than the original Python version.
We are 1.5 times faster than the NumPy version.


In [31]:
timeit_dict['Optimized contiguous'] = timeit_dict['Optimized']
print(timeit_dict)

{'Original python': 40.592430594, 'Numpy': 0.42193463800003883, 'Optimized': 0.2900572280000233, 'Compiled with cython': 29.361839215666578, 'Optimized typed': 33.45317457333332, 'Optimized memoryview': 0.8803729133333794, 'Optimized contiguous': 0.2900572280000233}


In [32]:
# medium file
print_report(main, input_file='bottle_medium.dat')
timeit_middle['Optimized contiguous'] = timeit_dict['Optimized']

Heat equation solver
Simulation finished in 0.2749786376953125 s
Heat equation solver
Simulation finished in 0.2616569995880127 s
Heat equation solver
Simulation finished in 0.2823629379272461 s
695 ms ± 18.5 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)

Current run_time: 0.6946752023 s
We are 58.4 times faster than the original Python version.
We are 1.6 times slower than the NumPy version.


In [33]:
# large file
print_report(main, input_file='bottle_large.dat')
timeit_large['Optimized contiguous'] = timeit_dict['Optimized']

Heat equation solver
Simulation finished in 1.1827707290649414 s
Heat equation solver
Simulation finished in 1.170565128326416 s
Heat equation solver
Simulation finished in 1.1820869445800781 s
2.34 s ± 18 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)

Current run_time: 2.3403195583 s
We are 17.3 times faster than the original Python version.
We are 5.5 times slower than the NumPy version.


# Instructor solution

In [34]:
%%cython
import time
import numpy as np
import matplotlib.pyplot as plt
import cython
cimport numpy as cnp



@cython.boundscheck(False)
@cython.wraparound(False)
@cython.cdivision(True)
@cython.profile(True)
cdef evolve(cnp.ndarray[cnp.double_t, ndim=2] u, 
            cnp.ndarray[cnp.double_t, ndim=2] u_previous,
            double a, double dt, double dx2, double dy2):
    """Explicit time evolution.
       u:            new temperature field
       u_previous:   previous field
       a:            diffusion constant
       dt:           time step. """

    cdef int n = u.shape[0]
    cdef int m = u.shape[1]

    cdef int i,j

    # Multiplication is more efficient than division
    cdef double dx2inv = 1. / dx2
    cdef double dy2inv = 1. / dy2

    for i in range(1, n-1):
        for j in range(1, m-1):
            u[i, j] = u_previous[i, j] + a * dt * ( \
             (u_previous[i+1, j] - 2*u_previous[i, j] + \
              u_previous[i-1, j]) * dx2inv + \
             (u_previous[i, j+1] - 2*u_previous[i, j] + \
                 u_previous[i, j-1]) * dy2inv )
    u_previous[:] = u[:]

def iterate(field, field0, a, dx, dy, timesteps, image_interval):
    """Run fixed number of time steps of heat equation"""

    dx2 = dx**2
    dy2 = dy**2

    # For stability, this is the largest interval possible
    # for the size of the time-step:
    dt = dx2*dy2 / ( 2*a*(dx2+dy2) )    

    for m in range(1, timesteps+1):
        evolve(field, field0, a, dt, dx2, dy2)
        if m % image_interval == 0:
            write_field(field, m)

def init_fields(filename):
    # Read the initial temperature field from file
    field = np.loadtxt(filename)
    field0 = field.copy() # Array for field of previous time step
    return field, field0

def write_field(field, step):
    plt.gca().clear()
    plt.imshow(field)
    plt.axis('off')
    plt.savefig('heat_{0:03d}.png'.format(step))

def main(input_file='bottle.dat', a=0.5, dx=0.1, dy=0.1, 
         timesteps=200, image_interval=4000):

    # Initialise the temperature field
    field, field0 = init_fields(input_file)

    print("Heat equation solver")
    # print("Diffusion constant: {}".format(a))
    # print("Input file: {}".format(input_file))
    # print("Parameters")
    # print("----------")
    # print("  nx={} ny={} dx={} dy={}".format(field.shape[0], field.shape[1],
    #                                          dx, dy))
    # print("  time steps={}  image interval={}".format(timesteps,
    #                                                      image_interval))

    # Plot/save initial field
    write_field(field, 0)
    # Iterate
    t0 = time.time()
    iterate(field, field0, a, dx, dy, timesteps, image_interval)
    t1 = time.time()
    # Plot/save final field
    write_field(field, timesteps)

    print("Simulation finished in {0} s".format(t1-t0))
    


In [35]:
print_report(main)
timeit_dict['Instructor solution'] = timeit_dict['Optimized']

Heat equation solver
Simulation finished in 0.05717182159423828 s
Heat equation solver
Simulation finished in 0.03509926795959473 s
Heat equation solver
Simulation finished in 0.06824278831481934 s
273 ms ± 29 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)

Current run_time: 0.2733422147 s
We are 148.5 times faster than the original Python version.
We are 1.5 times faster than the NumPy version.


In [36]:
print_report(main, input_file='bottle_medium.dat')
timeit_middle['Instructor solution'] = timeit_dict['Optimized']

Heat equation solver
Simulation finished in 0.36615610122680664 s
Heat equation solver
Simulation finished in 0.3675820827484131 s
Heat equation solver
Simulation finished in 0.39919185638427734 s
806 ms ± 16.5 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)

Current run_time: 0.8061036307 s
We are 50.4 times faster than the original Python version.
We are 1.9 times slower than the NumPy version.


In [37]:
print_report(main, input_file='bottle_large.dat')
timeit_large['Instructor solution'] = timeit_dict['Optimized']

Heat equation solver
Simulation finished in 1.389890193939209 s
Heat equation solver
Simulation finished in 1.5202710628509521 s
Heat equation solver
Simulation finished in 1.4495270252227783 s
2.59 s ± 75.4 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)

Current run_time: 2.5939087933 s
We are 15.6 times faster than the original Python version.
We are 6.1 times slower than the NumPy version.


# Display all the execution times

In [38]:
def display_results(x: dict()):
    v0 = list(x.values())[0]
    for k, v in x.items():
        msg = '{:<25s}: {:.3f}'.format(k,v)
        if k != 'Original python':
            msg += ' ({:.1f}X)'.format(v0/v)
        print(msg)

In [39]:
#del timeit_dict['Optimized']
display_results(timeit_dict)

Original python          : 40.592
Numpy                    : 0.422 (96.2X)
Optimized                : 2.594 (15.6X)
Compiled with cython     : 29.362 (1.4X)
Optimized typed          : 33.453 (1.2X)
Optimized memoryview     : 0.880 (46.1X)
Optimized contiguous     : 0.290 (139.9X)
Instructor solution      : 0.273 (148.5X)


In [40]:
display_results(timeit_middle)

Original python          : 282.861
Numpy                    : 2.323 (121.8X)
Optimized contiguous     : 0.695 (407.2X)
Instructor solution      : 0.806 (350.9X)


In [41]:
display_results(timeit_large)

Original python          : 1046.111
Numpy                    : 9.146 (114.4X)
Optimized contiguous     : 2.340 (447.0X)
Instructor solution      : 2.594 (403.3X)
